# Code demonstrating basic non-linear reduced rank regression

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg
import torch

from janelia_core.ml.non_linear_rr_regression import NonLinearRRRegresion

## Setup ground truth model

In [3]:
d_in, d_out, d_latent, n_smps = 10, 1, 5, 10000

In [4]:
true_model = NonLinearRRRegresion(d_in, d_out, d_latent)
true_model.init_weights()

## Generate data

In [5]:
x = 10*torch.rand([n_smps, d_in])
y = true_model.generate(x)

## Create model we will fit to data

In [6]:
fitted_model = NonLinearRRRegresion(d_in, d_out, d_latent)
fitted_model.init_weights()

## Fit model to data

### Setup optimizer

In [7]:
optimizer = torch.optim.SGD(fitted_model.parameters(), lr=.000001)

### Fit the model

In [8]:
for i in range(100000):
    optimizer.zero_grad()
    mns = fitted_model(x.data)
    nll = fitted_model.neg_log_likelihood(y.data, mns)
    nll.backward()
    optimizer.step()
    if i % 10000 == 0: 
        print(str(i) + ': ' + str(nll))

0: tensor(676.7207, grad_fn=<ThAddBackward>)
10000: tensor(-1997.0679, grad_fn=<ThAddBackward>)
20000: tensor(-1997.1992, grad_fn=<ThAddBackward>)
30000: tensor(-1997.4355, grad_fn=<ThAddBackward>)
40000: tensor(-1997.6963, grad_fn=<ThAddBackward>)
50000: tensor(-1997.9697, grad_fn=<ThAddBackward>)
60000: tensor(-1998.2139, grad_fn=<ThAddBackward>)
70000: tensor(-1998.3999, grad_fn=<ThAddBackward>)
80000: tensor(-1998.5483, grad_fn=<ThAddBackward>)
90000: tensor(-1998.6914, grad_fn=<ThAddBackward>)


## Standardize fitted model

In [9]:
#fitted_model.standardize()

In [10]:
%matplotlib qt

In [11]:
# plt.plot(true_model.v.detach().numpy())

In [18]:
cmp_a = 'w0'

true_p = getattr(true_model, cmp_a).detach().numpy()
est_p = getattr(fitted_model, cmp_a).detach().numpy()

print('True:' + str(true_p))
print('Estimated:' + str(est_p))

True:[[ 1.2590263  -1.2053972   1.0192255  -0.07308094 -0.26994985]
 [-0.27447778 -0.66325015 -0.7861464   0.31014583  1.5234082 ]
 [ 2.3829527   0.25622398  2.7640421   0.83891886  0.93794334]
 [-0.3133924  -0.02407174  0.48695427 -0.5916968  -0.58193713]
 [ 0.34021762 -2.0596166   1.6914777   0.715947    0.99405986]
 [-0.06310934  0.7603012  -1.1970751  -1.4510733  -1.3762658 ]
 [ 0.74086326 -0.85661495 -0.4107939   0.3653844  -0.0554444 ]
 [ 1.0266626  -0.2076049   0.16687714 -0.64381164  1.6740636 ]
 [ 0.4389416   0.22841504 -1.7320949  -0.918125   -0.11705229]
 [-0.09034348 -1.3651582  -1.3821507   0.19972941  0.12244542]]
Estimated:[[ 0.11375403 -0.8313391  -0.443484   -0.31506518  0.4224595 ]
 [-0.18684912  0.42683733 -0.55218244 -0.17781012 -0.9466747 ]
 [ 0.57588804  0.16899127 -0.16189727  0.3720263   0.45424137]
 [-2.5420985  -0.10009155 -1.2288768   0.9684696   0.5528205 ]
 [ 1.2885051  -0.149726   -0.8331857  -0.41344592 -1.0360056 ]
 [ 0.94741935 -0.92330265  0.23068245 -

In [ ]:
fitted_model.neg_log_likelihood(y.detach(), mns)

In [ ]:
torch.diag(true_model.g)

In [ ]:
a = torch.randn([2, 3])

In [ ]:
b = torch.from_numpy(np.asarray([2.0, 3, 4], dtype=np.float32))

In [ ]:
a/b

In [ ]:
a

In [ ]:
torch.sum((a**2)/b)